In [15]:
import chainlit as cl

from dotenv import load_dotenv # type: ignore
import os
import openai

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain.schema.runnable.config import RunnableConfig
from langchain.tools.retriever import create_retriever_tool

from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

from PyPDF2 import PdfReader
from langchain.schema import Document

from typing import Tuple, List, Optional

from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.graph_transformers import LLMGraphTransformer

#getting chat history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import GraphCypherQAChain

import logging

# Warning control
import warnings
warnings.filterwarnings("ignore")

from io import BytesIO


from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [ ]:
import os
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


In [2]:
load_dotenv('.env', override=True)
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_API_VERSION = os.getenv('AZURE_OPENAI_API_VERSION')
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_MODEL = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_MODEL')
NEO4J_URI=os.getenv("NEO4J_URI")
NEO4J_USERNAME=os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD=os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE=os.getenv("NEO4J_DATABASE")

In [12]:
graph = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [13]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

NameError: name 'GraphCypherQAChain' is not defined

In [4]:
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME_MODEL,
    api_version=AZURE_OPENAI_API_VERSION,
    api_key=OPENAI_API_KEY,
    temperature=0.3
)

In [5]:
embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002",
                                    azure_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
                                    azure_endpoint=AZURE_OPENAI_ENDPOINT,
                                    api_key=OPENAI_API_KEY
                                    )

In [6]:
vector_index = Neo4jVector.from_existing_graph(
        embedding=embeddings,
        search_type="hybrid",
        node_label="Document",
        text_node_properties=["text"],
        embedding_node_property="embedding",
        index_name='neo4j'
    )

2024-08-27 12:38:00 - HTTP Request: POST https://dialog-crm-westus.openai.azure.com//openai/deployments/ada2/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"


In [7]:
retriever=vector_index.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [8]:
from langchain.tools.retriever import create_retriever_tool

tool_search = create_retriever_tool(
    retriever=retriever,
    name="knowledge_grapg_search",
    description="Retrieve relevant info from a kwoledge graph that contains documents related to agents, prompt engineering, and adversarial attacks.",
)

In [10]:

prompt = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

In [11]:
# only creates the logical steps for us
react_agent = create_react_agent(llm, [tool_search], prompt)

AttributeError: 'str' object has no attribute 'input_variables'

In [ ]:
# executes the logical steps we created
agent_executor = AgentExecutor(
agent=react_agent, 
tools=[tool],
verbose=True,
handle_parsing_errors=True,
max_iterations = 5 # useful when agent is stuck in a loop
)

In [ ]:
query = "Which country has the highest budget?"
agent_executor.invoke({"input": query}) 

In [18]:
@tool
def retrieve_documents(query: str) -> list:
    """Retrieve documents from the vector store based on the query."""
    return retriever.invoke(query)

In [19]:
primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            " You are a helpful assistant tasked with answering user questions using the provided vector store. "
            " Use the provided vector store to retrieve documents. Then grade them to ensure they are relevant before answering the question. ",
        ),
        ("placeholder", "{messages}"),
    ]
)

In [9]:
from langchain.tools.retriever import create_retriever_tool

vectorstore_search = create_retriever_tool(
    retriever=retriever,
    name="knowledge_grapg_search",
    description="Retrieve relevant info from a kwoledge graph that contains documents related to agents, prompt engineering, and adversarial attacks.",
)

In [12]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [13]:
# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You are equipped with special graph database of information about agents prompt engineering and adversarial attacks.
If the query covers the topics of agents, prompt engineering or adversarial attacks, use the graph database search.
"""

# Prompt
prompt = ChatPromptTemplate.from_template("{input}")

# Create the ReAct agent
agent = create_cohere_react_agent(
    llm=llm,
    tools=[vectorstore_search],
    prompt=prompt,
)

In [14]:
agent_executor = AgentExecutor(
    agent=agent, tools=[vectorstore_search], verbose=True
)

In [15]:
agent_executor.invoke(
    {
        "input": "gsm associate with?",
        "preamble": preamble,
    }
)



> Entering new AgentExecutor chain...


TypeError: Completions.create() got an unexpected keyword argument 'raw_prompting'